In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objects as go


from CoolProp.CoolProp import PropsSI

import sklearn.metrics
import scipy.integrate as integrate
import scipy.optimize as sco

import test_general as tg

import fluids as fds
import ht 

import plot_functions_here as pfun

import general as gen
import model as ty
import proc as pr
import heat_transfer as bht
# importing sys
import sys
sys.path.insert(0,r'C:\Users\valen\Dropbox (DualSun)\PC\Documents\GitHub\RD-systems-and-test-benches\thermal')
import SK_standard as sk

In [2]:
input_name = r'\Inputs_V4.41-NI-glass-glass.xlsx'
hyp_name = r'\Model_hypotheses.xlsx'
data_path = r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Steady-state conditions inputs'
data_name = r'\221223_TUV_test_conditions_V4.41GGA_MPP.xlsx'

par = pr.import_input(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Inputs',input_name)
hyp = pr.create_dict_from_excel(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Hypotheses',hyp_name,'Main')
condi = pr.create_inputs_from_excel(data_path+data_name,par,hyp)

{'exchanger': {'name': nan, 'eta_nom': 0.201, 'Eff_T': -0.0034000000000000002, 'T_ref': 298.15, 'Eff_G': 0, 'G_ref': 1000, 'X_corr': 1, 'tau_g': 0.8, 'alpha_g': 0.05, 'eps_PV': 0.9, 'eps_g': 0.9, 'eps_bs': 1, 'w_pan': 1.134, 'L_pan': 1.708, 'orientation': 'portrait', 'lambd_upper_glass': 0.002, 'lambd_upper_EVA': 0.000515, 'lambd_si': 0.00015, 'lambd_lower_EVA': 0.000515, 'lambd_PVDF': 0.0, 'lambd_PET': 0.0, 'lambd_adh': 0.0, 'lambd_lower_glass': 0.002, 'k_glass': 0.8, 'k_EVA': 0.24, 'k_si': 60, 'k_PVDF': 0.2, 'k_PET': 0.3, 'k_adh': 0.2, 'L_abs': 1.342, 'w_abs': 0.976, 'geometry': 'harp', 'N_harp': 300, 'N_meander': 1, 'L_tube': 1.342, 'tube_geometry': 'rectangular', 'H_tube': 0.002, 'w_tube': 0.00281, 'W': 0.00335, 'l_c': 0.003, 'p_ext_tube': 0.00335, 'p_ext_tube_rad': 0.00335, 'lambd_air': 0.0, 'lambd_conductive_plate': 0.0, 'lambd_abs': 0.00035, 'lambd_riser_plate': 0.0, 'lambd_riser_back': 0.00035, 'lambd_ins': 0.00011999999999999999, 'eps_hx_top': 0, 'eps_hx_back': 0.8, 'eps_ins':

In [3]:
i=0
par_p = {'G':condi["G"][i],"T_amb":condi["T_amb"][i],"T_back":condi["T_amb"][i],"u":condi["u"][i], "u_back" : condi["u_back"][i], "T_fluid_in0":condi["T_fluid_in"][i]}
par_p["mdot"] = condi["mdot"][i]
ty.change_T_sky(par_p,hyp,'TUV')  # calculate Gp and T_sky

In [4]:
par_p["G"] = 0

In [5]:
def eta_PV(par,par_p,T):

    return par["eta_nom"]*par["X_rad"]*par["X_corr"]*(1+par["Eff_T"]*(T[PV.id]-par["T_ref"]))

In [6]:
amb = tg.Node(0, True, par_p["T_amb"])
sky = tg.Node(1, True, par_p["T_sky"])
glass = tg.Node(2, False)
glass.source(lambda T : par["alpha_g"]*par_p["G"])
PV = tg.Node(3, False)
PV.source(lambda T : par["tau_g"]*par_p["G"]*(1-eta_PV(par,par_p,T)))
Base = tg.Node(4, False)
absfin = tg.Node(5, False)
Base_ins = tg.Node(6, False)
absfin_ins = tg.Node(7, False)
tube = tg.Node(8, False)
tube_ins = tg.Node(9, False)
fluid = tg.Node(10, False)
back = tg.Node(11, True, par_p["T_back"])
back_rad = tg.Node(12, True, par_p["T_back"])

nodes = [amb, sky, glass, PV, Base, absfin, Base_ins, absfin_ins, tube, tube_ins, fluid, back, back_rad]

In [7]:
def h_top(par,par_p,T):
    return 5.

def h_rad(par,par_p,T):
    return 1.

def h_back_abs(par,par_p,T):
    return 3.

def h_rad_back(par,par_p,T):
    return 2.

def Fp(par,par_p,T):
    
    a1 = (1/(h_top(par,par_p,T)+h_rad(par,par_p,T)+1/par["R_g"]))*(1/par["R_g"])
    a2 = (h_top(par,par_p,T)+h_rad(par,par_p,T))*a1
    return 1/(a2*par["R_inter"]+h_rad(par,par_p,T)*par["R_inter"]+1)

def b(par,par_p,T):
    """Calculates the b factor and stores it in var["b"]
    
    $$
    b = S+h_{rad}T_{sky}+\frac{T_{amb}}{R_t}+\frac{T_{back}}{R_bF'}+\frac{h_{rad,f}T_{tube,mean}}{F'}
    $$
    
    Args:
        par (dict): dictionary containing the parameters
        par_p (dict): dictionary containing the meteo inputs
        var (dict): dictionary containing the variables
        
    Returns:
        None"""

    T_back = par_p["T_back"]
    R_b = par["R_2"] + 1/(h_back_abs(par,par_p,T)+h_rad_back(par,par_p,T))

    a0 = (1/(h_top(par,par_p,T)+h_rad(par,par_p,T)+1/par["R_g"]))*(par["alpha_g"]*par_p["G"] + h_top(par,par_p,T)*par_p["T_amb"] + h_rad(par,par_p,T)*par_p["T_sky"])
    a3 = - (par["alpha_g"]*par_p["G"] - h_top(par,par_p,T)*(a0 - par_p["T_amb"]) - h_rad(par,par_p,T)*(a0 - par_p["T_sky"]))
    S_star = par["tau_g"]*par_p["G"]*(1-eta_PV(par,par_p,T)) - a3 + h_rad(par,par_p,T)*par_p["T_sky"]

    return S_star + T_back/(R_b*Fp(par,par_p,T))

def j(par,var,T):
    """Calculates the j factor and stores it in var["j"]
    
    $$
    j = \frac{1}{R_{inter}F'}+\frac{1}{R_bF'}-\frac{1}{R_{inter}}+\frac{h_{rad,f}}{F'}
    $$
    
    Args:
        par (dict): dictionary containing the parameters
        var (dict): dictionary containing the variables
        
    Returns:
        None"""

    R_inter = par["R_inter"]
    R_b = par["R_2"] + 1/(h_back_abs(par,par_p,T)+h_rad_back(par,par_p,T))

    return 1/(Fp(par,par_p,T)*R_b) + 1/(R_inter*Fp(par,par_p,T)) - 1/R_inter

def m(par, var, T):
    """Calculates the m factor and stores it in var["m"]
    
    $$
    m = \sqrt{\frac{F'j}{k_{abs}\lambda_{abs}}}
    $$
    
    Args:
        par (dict): dictionary containing the parameters
        var (dict): dictionary containing the variables
        
    Returns:
        None"""

    lambd_abs = par["lambd_abs"]
    k_abs = par["k_abs"]

    return math.sqrt((Fp(par,par_p,T)*j(par,var,T))/(k_abs*lambd_abs))

In [8]:
def Cp(par,par_p,T,hyp):
    T_m = T[fluid.id] # K

    p_fluid = hyp["p_fluid"] # bar
    glycol_rate = hyp["glycol_rate"] # %

    # return PropsSI('C','P', p_fluid*100000, 'T', T_m, f'INCOMP::{hyp["fluid"]}[{glycol_rate}]')
    return 4180

In [9]:
par = par["exchanger"]

In [10]:
def h_fluid(par,par_p,T,hyp):
    return 300.


In [11]:
par["A_slice"] = par["L_tube"]*par["W"]
S_glass_amb = par["A_slice"]
S_glass_sky = par["A_slice"]
S_glass_PV = par["A_slice"]
S_PV_Base = par["L_tube"]*par["l_B"]
S_PV_absfin = par["L_tube"]*par["L_af"]*2
S_Base_Base_ins = par["L_tube"]*par["l_B"]
S_absfin_absfin_ins = par["L_tube"]*par["L_af"]*2
S_Base_tube = par["L_tube"]*par["l_B"]
S_tube_tube_ins = par["L_tube"]*par["l_B"]
S_tube_back = par["p_ext_tube"]
S_tube_back_rad = par["p_ext_tube_rad"]

par["R_tube"] = 1/par["C_B"]

glass_amb = tg.Edge(glass, amb, None)
glass_amb.convection_or_radiation(lambda T : h_top(par,par_p,T),S_glass_amb)
glass_sky = tg.Edge(glass, sky, None)
glass_sky.convection_or_radiation(lambda T : h_rad(par,par_p,T),S_glass_sky)
glass_PV = tg.Edge(glass, PV, None)
glass_PV.conduction(par["R_g"],S_glass_PV)
PV_Base = tg.Edge(PV, Base, None)
PV_Base.conduction(par["R_inter"],S_PV_Base)
PV_absfin = tg.Edge(PV, absfin, None)
PV_absfin.conduction(par["R_inter"],S_PV_absfin)
Base_absfin = tg.Edge(Base, absfin, lambda T : par["L_tube"]*par["k_abs"]*par["lambd_abs"]*m(par,par_p,T)*((b(par,par_p,T)/j(par,par_p,T))-T[Base.id])*math.tanh(m(par,par_p,T)*par["L_af"]))
Base_Base_ins = tg.Edge(Base, Base_ins, None)
Base_Base_ins.conduction(par["R_2"],S_Base_Base_ins)
absfin_absfin_ins = tg.Edge(absfin, absfin_ins, None)
absfin_absfin_ins.conduction(par["R_2"],S_absfin_absfin_ins)
Base_tube = tg.Edge(Base, tube, None)
Base_tube.conduction(par["R_tube"],S_Base_tube)
tube_tube_ins = tg.Edge(tube, tube_ins, None)
tube_tube_ins.conduction(par["R_2"],S_tube_tube_ins)
tube_back = tg.Edge(tube, back, None)
tube_back.convection_or_radiation(lambda T : h_rad(par,par_p,T),S_tube_back)
tube_back_rad = tg.Edge(tube, back_rad, None)
tube_back_rad.convection_or_radiation(lambda T : h_rad(par,par_p,T),S_tube_back_rad)

L_tube = par["L_tube"]
fluid.equation(lambda T : T[fluid.id] - T[Base.id] - ((T[Base.id]-par_p["T_fluid_in0"])/((-par["N_harp"]/(par_p["mdot"]*Cp(par,par_p,T,hyp)*(1/(h_fluid(par,par_p,T,hyp)*par["p_int_tube"]))))*L_tube))*math.exp((-par["N_harp"]/(par_p["mdot"]*Cp(par,par_p,T,hyp)*(1/(h_fluid(par,par_p,T,hyp)*par["p_int_tube"]))))*L_tube) - ((T[Base.id]+par_p["T_fluid_in0"])/((-par["N_harp"]/(par_p["mdot"]*Cp(par,par_p,T,hyp)*(1/(h_fluid(par,par_p,T,hyp)*par["p_int_tube"]))))*L_tube)))

In [12]:
edges = [glass_amb, glass_sky, glass_PV, PV_Base, PV_absfin, Base_absfin, Base_Base_ins, absfin_absfin_ins, Base_tube, tube_tube_ins, tube_back, tube_back_rad]

In [13]:
grap = tg.Graph(nodes,edges)

In [14]:
T_guess = np.array([node.temperature for node in grap.nodes])

In [15]:
par_p

{'G': 0,
 'T_amb': 303.99199999999996,
 'T_back': 303.99199999999996,
 'u': 0.4,
 'u_back': 0.0,
 'T_fluid_in0': 294.876,
 'mdot': 0.0608,
 'Gp': 4,
 'T_sky': 303.99199999999996}

In [16]:
for i,T in enumerate(T_guess):
    if T == 0.:
        T_guess[i] = (par_p["T_fluid_in0"]+par_p["T_amb"])/2 +5

In [25]:
T_guess[10] = 295

In [26]:
T_guess

array([303.992, 303.992, 304.434, 304.434, 304.434, 304.434, 304.434,
       304.434, 304.434, 304.434, 295.   , 303.992, 303.992])

In [31]:
grap.energy_balance(T_guess)

array([ 0.00000000e+00,  0.00000000e+00, -1.19225964e-02,  0.00000000e+00,
        1.21285827e-03, -1.21285827e-03,  0.00000000e+00,  0.00000000e+00,
       -2.96140000e-03,  0.00000000e+00,  1.21675117e+02,  0.00000000e+00,
        0.00000000e+00])

In [33]:
grap.solve(T_guess)

In [34]:
grap.show_temp()

Node 0 temperature: 303.99199999999996
Node 1 temperature: 303.99199999999996
Node 2 temperature: 303.99199999999996
Node 3 temperature: 303.99199999999996
Node 4 temperature: 303.99199999999996
Node 5 temperature: 303.99199999999996
Node 6 temperature: 303.99199999999996
Node 7 temperature: 303.99199999999996
Node 8 temperature: 303.99199999999996
Node 9 temperature: 303.99199999999996
Node 10 temperature: 172.98056139516413
Node 11 temperature: 303.99199999999996
Node 12 temperature: 303.99199999999996
